<a href="https://colab.research.google.com/github/frankausberlin/notebook-collection/blob/main/snippetpearls_lazy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _sp: lazy student


In [ ]:
#@markdown <font size='+5' color='purple'><b>lazy student</b></font><br>
#@markdown This is for the work-optimized student - openai's [prompt engineering](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) with access to collections of video-lecture transicriptions, focused on bilingualism and yaml persistence.
#@markdown * Choose your language, choose a collection - choose a course - select a video - build your chapters - clean it, prompt it, loop it - have fun...
#@markdown * The prompt functions needs an **[openai](https://platform.openai.com/account/usage)** account for send your gpt request direct out of the widget.
#@markdown * If you have **no account** you can still use it (no aichapters, gpt only as text). Use copy/paste and a **[public gpt](https://poe.com/)** instead.
#@markdown * It manages collections on your **gdrive** (~ 50 MB max) or **local runtime** - there are two examples (new ones can easily be added):
#@markdown > * ['Practical Deeplearning for Coders'](https://course.fast.ai/) by fastai (**folder dlfc**) and 
#@markdown > * A [collection by Stanford](https://www.youtube.com/@stanfordonline/playlists?view=50&sort=dd&shelf_id=4): CS221, CS224, CS229, CS330 (**folder stanford**).
#@markdown * A **collection** is a list of youtube **playlists** (playlists.yml) with its own **folder** for the **created yml** files.
#@markdown * More questions? **Look in the help tab or code** - there are more detailed descriptions.
#@markdown ---
####################################################################################################################################
# 
helptext="""Basic concepts:
* The yamls created 
  playlists.yml    : Infos and videos list for every playlist
  prompts.yml      : The prompts - each for english and xx
  <vid>_en.yml     : The english transcription
  <vid>_xx.yml     : The xx transcription
  <vid>_info.yml   : Description, chapters and aichapters for video
  <vid>_loops.yml  : Prompt loops history for video
* You have three persistence levels:
  - hosted runtime session only - stored in directory '/content/<collection>' 
    (not recommended) - just create a directory <collection> in your session 
    content folder before first run.
  - hosted runtime with gdrive - stored in the folder 
    '/gdrive/MyDrive/<collection> - recommended default setting.
  - local runtime in the folder 'content/<collection>' (for advanced)
    simply create the <collection> folders inside the folder 'content'.
* The persistence mechanism currently stores no simple prompts yet - you have 
  to do that yourself (copy/paste).
* Yamls are only generated when they are needed:
    - playlists.yaml               by first run
    - *_en, *_xx, *_info files     by selection of a video
    - prompts.yml                  by save 
    - *_loops.yml                  after doing loop.
* Without api-key for openai the features using the api are disabled 
  ('send'(request), 'loop' and 'build') but the prompt editor works.
* The 'aichapters' prompt is a system prompt and can not be deleted but modified
  with a 70 char limit used for response.
* The textareas does show the token count (the cyan text), the token factor 
  ('*'-textfield, editable), the request temperatur ('°'-textfield, editable).
* Select a prompt (green button) and a chapter -> click show-button to create 
  the prompt in the textarea and click send to send the textarea (!) as request 
  and wait for response.
* Between show and send you have the possibility to edit the prompt.  
* The loop mechanic takes the transcript from every chapter as input for the 
  selected prompt and show the response in the textarea - be carefull it cannot 
  be interupted (check token count in chapters - 4000 token max).
* The api is not very stable - **sometimes it hangs** (~ about 1-3 minutes)
* There are 3 attempts for requests - so mostly the result is given.
* If you want **change the translation language** just delete playlists.yml 
  and run the cell again.

Notes UI:
* When you run the cell it builds the widgets for the selecetd collection.
* You have a accordion tabed gridbox for every plalist and the prompt editor.
* On start it takes first video of first playlist and try to create the chapters
  out of the description.
* It creates the video radio buttons and the buttons to work with video:
    'all >>>>', 'clean', 'from description', 'auto', 'aichaps', 'build' 
  and the prompt-buttons
* There are important infos in tooltips:
  - 'all >>>>'-button: the original description of the video.
  - prompt-buttons: the prompt textes (en / xx)
  - all other buttons: the description what they do.
* There are are log message line below the accordion tab - shows current events.  
* With a small **editor** you have the possibility to have a little prompt 
  engineering **fun** (note the 4000 token limit).
* When you add a new prompt in the editor it will be shown as green prompt 
  button.

Notes code:
* The current select video / playlist is set in globals: function globalize().
* The request uses the textare text - you can reduce token len when you use
  the clean-button before send.
* The libs pytube, youtubesearchpython, youtube_transcript_api 
  (and openai if selected) will be installed.
"""
try:
  from pytube                   import  Playlist
  from youtubesearchpython      import  Video
  from youtube_transcript_api   import  YouTubeTranscriptApi  
except:
  !pip install pytube youtube-search-python youtube-transcript-api -q

import                                  os, yaml, getpass, time
from   ipywidgets               import  Accordion, VBox, HBox, Box, GridBox, HTML, Tab, Textarea, Text, RadioButtons, Button, Layout, Label
from   datetime                 import  datetime
from   pytube                   import  Playlist
from   youtubesearchpython      import  Video
from   youtube_transcript_api   import  YouTubeTranscriptApi  


###################################################defaults/playlists.yaml##########################################################
playlist_prefix                           = 'https://www.youtube.com/playlist?list='
country_code_for_the_translation_language = 'de'   #@param {type:"string"}
yes_i_have_this_chatgpt_openai_account    = False   #@param {type:"boolean"}
use_english_for_ai_auto_chapters          = True  #@param {type:"boolean"}
selected_collection                       = 'A collection of free Stanford courses: CS221, CS224, CS229, CS330' #@param ['Practical Deep Learning for Coders / fast.ai live coding & tutorials', "A collection of free Stanford courses: CS221, CS224, CS229, CS330"]

# The collections 
if 'Stanford' in selected_collection:     
  courses = { 'CS221:  2021 - Artificial Intelligence: Principles and Techniques (Percy Liang)'             : 'PLoROMvodv4rOca_Ovz1DvdtWuz8BfSWL2',
              'CS221:  2019 - Artificial Intelligence: Principles and Techniques (Percy Liang)'             : 'PLoROMvodv4rO1NB9TD4iUZ3qghGEGtqNX',
              'CS224N: 2021 - NLP with Deep Learning (Christopher Manning)'                                 : 'PLoROMvodv4rOSH4v6133s9LFPRHjEmbmJ',
              'CS224W: 2021 - Machine Learning with Graphs (Jure Leskovec)'                                 : 'PLoROMvodv4rPLKxIpqhjhPgdQy7imNkDn',
              'CS224U: 2021 - Natural Language Understanding (Christopher Potts)'                           : 'PLoROMvodv4rPt5D0zs3YhbWSZA8Q_DyiJ',
              'CS229:  2018 - Machine Learning Full Course (Andrew Ng)'                                     : 'PLoROMvodv4rMiGQp3WXShtMGgzqpfVfbU',
              'CS229:  2019 - Machine Learning Course (Anand Avati)'                                        : 'PLoROMvodv4rNH7qL6-efu_q2_bPuy0adh',
              'CS330:  2022 - Deep Multi-Task & Meta Learning - What is multi-task learning? (Chelsea Finn)': 'PLoROMvodv4rNjRoawgt72BBNwL2V7doGI' }
  folder_for_playlist_collection = 'stanford'

if 'Deep Learning for Coders' in selected_collection:
  courses = { 'Practical Deep Learning for Coders 2022'  : 'PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU',
              'Practical Deep Learning 2022 Part 2'      : 'PLfYUBJiXbdtRUvTUYpLdfHHp9a58nWVXP',
              'fast.ai live coding & tutorials'          : 'PLfYUBJiXbdtSLBPJ1GMx-sQWf6iNhb8mM',
              'Practical Deep Learning for Coders (2020)': 'PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM',
              'Practical Deep Learning for Coders 2019'  : 'PLfYUBJiXbdtSIJb-Qd3pw0cqCbkGeS0xn' } 
  folder_for_playlist_collection = 'dlfc'

#         ||          your collection here
#      \ \||/ /        
#       \ \/ /        (don't forget: add to param-list in line 100)
#        \  /         
#         \/          
"""
if 'some unique' in selected_collection: 
  courses = { 'Course 1'  : 'PL******',
              'Course 2'  : 'PL******',
              'Course 3'  : 'PL******'} 
  folder_for_playlist_collection = 'yourfolder'
"""

# change folder back if needed 
if os.getcwd().split('/')[-2] == 'content' or os.getcwd().split('/')[-2] == 'MyDrive': os.chdir('..') 

# change to selected folder
if not folder_for_playlist_collection in os.listdir() and not folder_for_playlist_collection in os.getcwd(): 
  # gdrive
  from google.colab import drive
  if not 'gdrive'  in os.listdir('/'):                    drive.mount('/gdrive')
  if not 'MyDrive' in os.getcwd():                        os.chdir ('/gdrive/MyDrive')
  if not folder_for_playlist_collection in os.listdir():  os.mkdir (folder_for_playlist_collection)
if folder_for_playlist_collection in os.listdir():        os.chdir (folder_for_playlist_collection)

# playlists.yml <-> playlists_yaml
if not os.path.exists('playlists.yml'):
  playlists_yaml = {}

  # playlist loop
  for i,title in enumerate(courses):
    # lazy using pytube
    from pytube import Playlist
    try:      pl     = Playlist(f'{playlist_prefix}{courses[title]}')
    except:   raise    Exception ('BIG_OOPS')
    try:      descr  = pl.description
    except:   descr  = 'no description in pytube'
    try:      links  = [link for link in pl]
    except:   links  = []
    videos = [{'title'   :f'Video {vnr+1}', 'url':link, 'info':f"{link.split('?v=')[-1]}_info.yml",
                'trans_en':f"{link.split('?v=')[-1]}_en.yml", 
                'trans_xx':f"{link.split('?v=')[-1]}_{country_code_for_the_translation_language[:2]}.yml"
              } for vnr, link in enumerate(links)]
    playlists_yaml[courses[title]] = {'title':title, 'url':f'{playlist_prefix}{courses[title]}','description':descr, 'videos':videos}
  # write playlists.yml
  yaml.dump (playlists_yaml, open('playlists.yml', 'w'))

# read playlists.yml
else:
  playlists_yaml = yaml.load(open('playlists.yml', 'r'), Loader=yaml.FullLoader)


###########################################################helper###################################################################
def prettyMaking (trans):
  """ make the transcription text format for the textareas """
  txt = ''
  for all in trans: 
    tmp, block, sec = all['text'].replace('\xa0','').replace('\n',''), [], int(all['start'])
    h, i = sec//3600, len (tmp)
    m    = (sec - (h*3600)) // 60
    s    = sec - h*3600 - m*60
    # make a text block
    while len (tmp) > 70:
      for i in reversed(range(70)):
        if tmp[i] == ' ': break
      block.append(tmp[:i])
      tmp = tmp[i+1:]
    block.append(tmp[:i])
    # and show it with tabulators
    for i,l in enumerate (block):
      if i == 0: txt += f"{h}:{m:02d}:{s:02d}\t{l}\n"
      else:      txt += f"\t{l}\n"
  return txt

def getStartEndFromChapterButton (b):
  gb_main                   = mainAccordion.children[mainAccordion.selected_index]
  vb_vsel, vb_chap          = gb_main.children[0], gb_main.children[1]
  bx_csel, hb_chcf, vb_view = vb_chap.children[0], vb_chap.children[1], vb_chap.children[2]

  # start ts from button
  ts = b.description.split(' ')[0]
  if len(ts.split(':')) == 2: start = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: start = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])
  
  # search next button
  nb = None
  for i,sb in enumerate(bx_csel.children):
    if b.description == sb.description: break

  # end ts from search
  if i >= len(bx_csel.children)-1: ts = '99:00:00'
  else: ts = bx_csel.children[i+1].description.split(' ')[0]
  if len(ts.split(':')) == 2: end = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: end = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])
  return start, end

def buildTranscriptionByButtonTimestamp (b, vid):
  # start ts from button
  ts = b.description.split(' ')[0]
  if len(ts.split(':')) == 2: start = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: start = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])
  
  # search next button
  nb = None
  for i,sb in enumerate(bx_csel.children): 
    if b.description == sb.description: break

  # end ts from search
  if i >= len(bx_csel.children)-1: ts = '99:00:00'
  else: ts = bx_csel.children[i+1].description.split(' ')[0]
  if len(ts.split(':')) == 2: end = int(ts.split(':')[0])*60+int(ts.split(':')[1])
  else: end = int(ts.split(':')[0])*3600+int(ts.split(':')[1])*60+int(ts.split(':')[2])

  # build part for view
  entr = [t for t in enTrans[vid] if t['start'] >= start and t['start'] < end]
  xxtr = [t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end]

  return entr, xxtr

def makeAutoChapters (max,tmp):
  # generate chapters with not more than max tokens
  chapters = []
  safety=10000
  while(True):
    if safety < 0: break
    safety -= 1
    if len (chapters) > 0 and chapters[-1] == tmp[0].split('\t')[0]: break
    chapters.append(tmp[0].split('\t')[0])
    grow, lastTS = '', 0
    for i,line in enumerate(tmp):
      if line != '' and line[1] == ':':
        lastTS = i
      if len(grow+line+'\n')//3.1 < max:
        grow = grow + line + '\n'
      else:
        break
    tmp = tmp[lastTS:]
  return chapters

def taCleaner (lines,tsCount,bs=None):
  org, tmp, trigger = tsCount, '', False
  for i,l in enumerate(lines):
    if tsCount and not i%(len(lines)//org):
      tsCount -= 1
      trigger = True
    if trigger and l != '' and l[1] == ':':
      if i > 0: tmp += '\n'
      tmp += l.split('\t')[0]+'\n' 
      trigger = False
    if '\t' in l: 
      tmp += l.split('\t')[1]+'\n'
  # return tmp - if no block wanted   
  if bs == None: bs = 60
  lines, tmp2, nl = tmp.split('\n'), '', ''
  for i,l in enumerate(lines):
    if len (l) > 1 and l[1] != ':':
      words = l.split(' ')
      for w in words:
        if len (nl+' '+w) < bs: 
          nl += ' '+w
        else:
          tmp2 += nl + '\n'
          nl = w
    else:
      tmp2 += l+'\n'
  return tmp2.replace('\n ','\n')

def storeLoop (vid, txt):
  if txt[:13] == '... do prompt':
    if not os.path.exists(f'{vid}_loops.yml'): loops_yaml = {}
    else: loops_yaml = yaml.load (open(f'{vid}_loops.yml', 'r'), Loader=yaml.FullLoader)
    loops_yaml[datetime.now().strftime("%Y%m%d-%H%M%S")] = txt
    yaml.dump (loops_yaml, open(f'{vid}_loops.yml', 'w'))



########################################################openai-stuff################################################################
# defaults 
summary_20_en = """Below is an excerpt from a transcription of a video that starts after this string '##x##'. 
The video is a course about Deep Learning. 
Create a summary of the content of the excerpt in 20 words or less. 
To do this, first clean up the raw text by removing the time stamps, merging the text, and ridding it of errors or clutter.
##x##"""
summary_20_xx = """Im Folgenden findest du einen Auszug aus der Transkription eines Videos, das nach dieser Zeichenfolge '##x##' beginnt. 
Das Video ist ein Kurs über Deep Learning. 
Erstelle eine Zusammenfassung des Inhalts des Auszugs in 20 Wörtern oder weniger. 
Bereinige dazu zunächst den Rohtext, indem du die Zeitstempel entfernst, den Text zusammenführst und ihn von Fehlern oder Unordnung befreien.
##x##"""
keywords_en = """The text is an excerpt from the transcription of a video and it starts after this string: '##x##'.
The video is one of many in a tutorial series on Deep Learning for programmers.
Create a list of three, four, or five keywords that describe the text - depending on the length or information content of the text.
The keywords are specifically intended to give software developers a thematic overview of the content of the text.
'##x##'
"""
keywords_xx = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: '##x##'.
Das Video ist eines von vielen aus einer Tutorial Reihe zum Thema Deep Learning für Programmierer.
Erzeuge eine Liste mit drei, vier oder fünf Schlagwörtern, die den Text beschreiben - jeh nach Länge oder Informationsgehalt des Textes.
Die Schlagwörter sollen speziell Softwareentwickler einen thematischen überblick über den Inhalt des Textes liefern.
'##x##'
"""
aichapters_xx = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: '##x##'.
Das Video ist eines von vielen aus einer Tutorial Reihe zum Thema Deep Learning für Programmierer.
Finde einen Titel für diese Sektions des Videos, der zusammenfassende Schlagwörter beinhaltet und nicht mehr als 10 Wörter sein soll.
Gib den nur Titel als ergebnis zurück.
'##x##'
"""
aichapters_en = """The text is an excerpt from the transcription of a video and it starts after this string: '##x##'.
The video is one of many in a tutorial series on Deep Learning for programmers.
Find a title for this section of the video that includes summary keywords and should be no more than 10 words.
Return only the title as the result.
'##x##'
"""
hints_xx = """Bei dem Text handelt es sich um einen Auszug aus der Transkription eines Videos und er beginnt nach dieser Zeichenfolge: ##x##.
Das Video ist eines von vielen aus einer Tutorial Reihe zum Thema Deep Learning für Programmierer.
Der Sprecher in dem video gibt viele Hinweise und Tipps zu verschiedenen Techniken bzw. Methoden.
Erstelle eine liste der Hinweise und Tipps die der Sprecher gibt. 
Angaben eingeschlossen in < und > innerhalb eines Strukturaufbaus sind variablen und sollen entsprechend ihrer bezeichung ersetzt werden.
Die Liste soll folgenden Strukturaufbau haben (Strukturaufbau eingeschlossen in <<< und >>>):
<<<
<leerzeile>
<Bezeichnung Hinweis>
<leerzeile>
Subjekt: <subjekt>
<Zusammenfassung Hinweis>
>>> 
Hier ist noch ein konkretes Beispiel zum besseren Verständnis (Beispiel eingeschlossen in <<< und >>>):
<<<

Schlüsselbibliotheken die man kennen muss

Subjekt: Numpy, Pandas, Matplotlib, Pytorch
Es werden die Bibliotheken Numpy, Pandas, Matplotlib und Pytorch kurz vorgestellt und ihre Wichtigkeit betont.
>>>
##x##
"""
hints_en = """The text is an excerpt from the transcription of a video and it starts after this string: ##x##.
The video is one of many in a tutorial series on Deep Learning for programmers.
The speaker in the video gives many hints and tips on various techniques or methods.
Make a list of the hints and tips the speaker gives. 
The information enclosed in < and > within a structure-block are variables and should be replaced corresponding to their meaning.
The list should have the following structure-block (structure-block enclosed in <<< and >>>):
<<<

<name hint>

Subject: <subject>
<summary note>
>>> 
Here is another concrete example for better understanding (example enclosed in <<< and >>>):
<<<

Key libraries you need to know

Subject: numpy, pandas, matplotlib, pytorch.
The libraries Numpy, Pandas, Matplotlib and Pytorch are briefly introduced and their importance is emphasized.
>>>
##x##
"""
if yes_i_have_this_chatgpt_openai_account:
  default_model       = 'gpt-3.5-turbo'
  default_temperature = 0.3
  #myApiKey            = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
  try:    import openai
  except: 
    !pip install -q openai 
  import openai
  try:     myApiKey = os.environ["OPENAI_API_KEY"]
  except:  pass  
  try:     os.environ["OPENAI_API_KEY"] = openai.api_key = myApiKey 
  except:  os.environ["OPENAI_API_KEY"] = openai.api_key = getpass.getpass (prompt='Your API key: ')
  display(HTML("""<font color='green'>Set API-Key done!</font>"""))

# prompts_yaml
if not os.path.exists('prompts.yml'):
  prompts_yaml = {'summary 20': {'en':summary_20_en,'xx':summary_20_xx}, 
                  'keywords':   {'en':keywords_en,'xx':keywords_xx},
                  'hints':      {'en':hints_en,'xx':hints_xx},
                  'aichapters': {'en':aichapters_en,'xx':aichapters_xx}}
  yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
else:
  prompts_yaml = yaml.load(open('prompts.yml', 'r'), Loader=yaml.FullLoader)

def get_completion(prompt, model=None,temperature=None):
  # do an openai api call
  if not model:         model         = default_model
  if not temperature:   temperature   = default_temperature
  messages, ret = [{"role": "user", "content": prompt}], ''
  try: # first
    time.sleep (1)
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature)
    return response.choices[0].message["content"]
  except:
    pass
  try: # second
    time.sleep (2)
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature)
    return response.choices[0].message["content"]
  except:
    pass
  try: # last
    time.sleep (3)
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=temperature)
    return response.choices[0].message["content"]
  except:
    return 'oops - something goes wrong...'

def promptButtonClick (b):
  # set prompt actve -> show-button
  hb_encf.children[1].value, hb_xxcf.children[1].value = b.description, b.description
  if 'info' in [b.button_style for b in [bu_desc,*bx_csel.children]]: bu_enSh.disabled = bu_xxSh.disabled = False
  bu_enLo.disabled = bu_xxLo.disabled = False

def enShowButtonClick (b):
  # generate prompts and show
  if len (ta_entr.value) > 50 and len (prompts_yaml[hb_encf.children[1].value]['en']) > 50:
    ta_entr.value, bu_enSe.disabled = f"{prompts_yaml[hb_encf.children[1].value]['en']}\n{ta_entr.value}", False

def xxShowButtonClick (b):
  # generate prompts and show
  if len (ta_xxtr.value) > 50 and len (prompts_yaml[hb_xxcf.children[1].value]['xx']) > 50:
    ta_xxtr.value, bu_xxSe.disabled = f"{prompts_yaml[hb_xxcf.children[1].value]['xx']}\n{ta_xxtr.value}", False

def enSendButtonClick (b):
  # show wait message / response
  prompt = hb_encf.children[1].value
  bu_enSh.disabled, bu_enSe.disabled, hb_encf.children[1].value, tmp = True, True, '<<<select prompt>>>', ta_entr.value
  ta_entr.value = '... waiting for response'
  ta_entr.value = f'>>>>> response for prompt {prompt} >>>>>\n'+get_completion (tmp,temperature=float(hb_encf.children[3].value))

def xxSendButtonClick (b):
  # show wait message / response
  prompt = hb_xxcf.children[1].value
  bu_xxSh.disabled, bu_xxSe.disabled, hb_xxcf.children[1].value, tmp = True, True, '<<<select prompt>>>', ta_xxtr.value
  ta_xxtr.value = '... waiting for response'
  ta_xxtr.value = f'>>>>> response for prompt {prompt} >>>>>\n'+get_completion (tmp,temperature=float(hb_xxcf.children[3].value))

def enLoopButtonClick (b):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  
  # buttonstyles / get video infos for selected video - using acIndex and rbIndex
  plid    = [courses[all] for all in courses][acIndex]
  video   = playlists_yaml[plid]['videos'][rbIndex]
  vid     = video['url'].split('?v=')[-1]

  # show wait message / response
  prompt = hb_encf.children[1].value
  ta_entr.value = '... do prompt "'+prompt+'" for ...\n'
  for b in bx_csel.children:
    ta_entr.value += '\n>>>>>>'+b.description + '>>>>>>\n'
    start, end    = getStartEndFromChapterButton (b)
    tmp           = taCleaner (prettyMaking ([t for t in enTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
    response      = get_completion ( f"{prompts_yaml[prompt]['en']}\n{tmp}", temperature=float(hb_encf.children[3].value) )
    ta_entr.value += f"{response}\n" if response[-1] != '\n' else f"{response}"

  # yaml it
  storeLoop (vid, ta_entr.value)
  ta_entr.value += f"\n\n>>>>>> write to {f'{vid}_loops.yml >>>>>>'}"

def xxLoopButtonClick (b):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  
  # buttonstyles / get video infos for selected video - using acIndex and rbIndex
  plid    = [courses[all] for all in courses][acIndex]
  video   = playlists_yaml[plid]['videos'][rbIndex]
  vid     = video['url'].split('?v=')[-1]

  # show wait message / response
  prompt = hb_xxcf.children[1].value
  ta_xxtr.value = '... do prompt "'+prompt+'" for ...\n'
  for b in bx_csel.children:
    ta_xxtr.value += '\n>>>>>>'+b.description + '>>>>>>\n'
    start, end    = getStartEndFromChapterButton (b)
    tmp           = taCleaner (prettyMaking ([t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
    response      = get_completion ( f"{prompts_yaml[prompt]['xx']}\n{tmp}", temperature=float(hb_xxcf.children[3].value) )
    ta_xxtr.value += f"{response}\n" if response[-1] != '\n' else f"{response}"

  # yaml it
  storeLoop (vid, ta_xxtr.value)
  ta_xxtr.value += f"\n\n>>>>>> write to {f'{vid}_loops.yml >>>>>>'}"


def text_change(_):
  cyanbox = lambda s: "<table width='50'><tr><td align='center'><p style='background-color:#0AFFFF'>"+s+"</p></td></tr></table>"
  # compute tokens
  hb_encf.children[0].value = cyanbox(str(int(len(ta_entr.value) // float(hb_encf.children[2].value)+1)))
  hb_xxcf.children[0].value = cyanbox(str(int(len(ta_xxtr.value) // float(hb_xxcf.children[2].value)+1)))

def refreshPrompts():
  # tab loop
  for gb_main in mainAccordion.children[:-1]:
    vb_vsel        = gb_main.children[0]
    prompt_buttons = [Button (description=p,
                              tooltip=str(prompts_yaml[p]['en'])+'\n'+str(prompts_yaml[p]['xx']),
                              style={'button_color':'lightgreen'}) for p in prompts_yaml if p != 'aichapters']
    for b in prompt_buttons:             b.on_click (promptButtonClick)
    firstPromptButtonPos = 3 # for safety
    for i,but in enumerate (vb_vsel.children):
      try: # somtimes python is ... - hm ... - special
        if but.style.button_color == 'lightgreen':
          firstPromptButtonPos = i
          break
      except: pass
    for oldbut in vb_vsel.children[firstPromptButtonPos:]:  del oldbut
    vb_vsel.children = [*vb_vsel.children[:firstPromptButtonPos],*prompt_buttons]

def prompt_add (_):
  # make new empty prompt
  rb_prompts.disabled, tx_promtit.value, tx_promtit.disabled = True, '', False
  bu_addprmt.disabled, hb_prompts.children[0].value, hb_prompts.children[1].value = True, '', ''

def prompt_save (_):
  # save yaml / refresh widgets
  prompts_yaml[tx_promtit.value] = {'en':hb_prompts.children[0].value,'xx':hb_prompts.children[1].value}
  rb_prompts.options  = tuple ([tx_promtit.value,*rb_prompts.options]) if not tx_promtit.value in rb_prompts.options else rb_prompts.options
  rb_prompts.disabled = bu_addprmt.disabled = False
  yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
  refreshPrompts()

def prompt_del (_):
  if tx_promtit.value == 'aichapters': return
  # remove prompt from rb-list / save yaml / trigger refresh prompt buttons
  if tx_promtit.value in rb_prompts.options:
    if tx_promtit.value in prompts_yaml: 
      del prompts_yaml[tx_promtit.value]
      yaml.dump (prompts_yaml, open('prompts.yml', 'w'))
    rb_prompts.options = tuple ([o for o in rb_prompts.options if o != tx_promtit.value])
  else: promptSelect(None)
  rb_prompts.disabled = bu_addprmt.disabled = False
  refreshPrompts()

def promptSelect (_):
  # set prompt title in en/xx areas
  hb_prompts.children[0].value, hb_prompts.children[1].value = prompts_yaml[rb_prompts.value]['en'], prompts_yaml[rb_prompts.value]['xx']
  tx_promtit.value, tx_promtit.disabled                      = rb_prompts.value, True


########################################################chapterbuttons##############################################################
def buttonClick (b):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  
  # buttonstyles / get video infos for selected video - using acIndex and rbIndex
  for but in bx_csel.children: but.button_style   = ''
  bu_desc.button_style, b.button_style            = '', 'info'
  plid    = [courses[all] for all in courses][acIndex]
  video   = playlists_yaml[plid]['videos'][rbIndex]
  vid     = video['url'].split('?v=')[-1]
  # show complete text
  if b.description == 'all >>>>':
    ta_entr.value = prettyMaking (enTrans[vid])
    ta_xxtr.value = prettyMaking (xxTrans[vid])
  # or chapter filtered
  else:
    start, end    = getStartEndFromChapterButton (b)
    ta_entr.value = prettyMaking ([t for t in enTrans[vid] if t['start'] >= start and t['start'] < end])
    ta_xxtr.value = prettyMaking ([t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end])
  # unselect prompt
  hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
  hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
  bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}

def fromDescriptionButtonClick (_):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  plid  = [courses[all] for all in courses][acIndex]
  video = playlists_yaml[plid]['videos'][rbIndex]
  vid   = video['url'].split('?v=')[-1]
  tmp   = prettyMaking(enTrans[vid]).split('\n')

  # del old chapter buttons
  for oldbut in bx_csel.children: del oldbut

  # build buttons
  chapters = videoInfos[vid]['chapters']
  bu_desc.tooltip = str(videoInfos[vid]['description'])
  bx_csel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                              tooltip      = c,
                              layout       = Layout(width='auto', height='21px')) 
                      for i,c in enumerate (chapters)]
  for button in bx_csel.children: button.on_click(buttonClick)

  # unselect prompt / clear textareas
  hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
  hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
  ta_entr.value, ta_xxtr.value = "... select chapter or click 'all >>>>'", '...'
  bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}

def autoButtonClick (_):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  plid  = [courses[all] for all in courses][acIndex]
  video = playlists_yaml[plid]['videos'][rbIndex]
  vid   = video['url'].split('?v=')[-1]
  tmp   = prettyMaking(enTrans[vid]).split('\n')

  # del old chapter buttons
  for oldbut in bx_csel.children: del oldbut

  # build buttons
  chapters, lastTS = makeAutoChapters (int(hb_Caut.children[0].value),tmp), 0
  bu_desc.tooltip = str(videoInfos[vid]['description'])
  bx_csel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                              tooltip      = c,
                              layout       = Layout(width='auto', height='21px')) 
                      for i,c in enumerate (chapters)]
  for button in bx_csel.children: button.on_click(buttonClick)

  # unselect prompt / clear textareas / get video infos for selected video - using acIndex and rbIndex
  ta_entr.value, ta_xxtr.value = "... select chapter or click 'all >>>>'", '...'
  hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
  hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
  bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}

def cleanButtonClick (_):
  # parse blocksize - check > 40 and < 200
  val = hb_Tcle.children[0].value
  if not ',' in val:  bs = None
  else:               bs = int(val.split(',')[1]) if int(val.split(',')[1]) < 200 and int(val.split(',')[1]) > 40 else None
  if "select chapter or click 'all >>>>'" in ta_entr.value: return
  # use taCleaner (<linelist>,<count timestamps>,<blocksize>)
  ta_entr.value = taCleaner ([l for l in ta_entr.value.split('\n') if l != ''], int(val.split(',')[0]), bs)
  ta_xxtr.value = taCleaner ([l for l in ta_xxtr.value.split('\n') if l != ''], int(val.split(',')[0]), bs)

def aiShowButtonClick (_):
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  plid  = [courses[all] for all in courses][acIndex]
  video = playlists_yaml[plid]['videos'][rbIndex]
  vid   = video['url'].split('?v=')[-1]
  tmp   = prettyMaking(enTrans[vid]).split('\n')

  # del old chapter buttons
  for oldbut in bx_csel.children: del oldbut

  # build ai chapter buttons
  chapters, lastTS = videoInfos[vid]['aichapters'] if 'aichapters' in videoInfos[vid] else [], 0
  for child in bx_csel.children: del child
  bx_csel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                              tooltip      = c,
                              layout       = Layout(width='auto', height='21px')) 
                      for i,c in enumerate (chapters)]
  for button in bx_csel.children: button.on_click(buttonClick)

  # unselect prompt / clear textareas
  ta_entr.value, ta_xxtr.value = "... select chapter or click 'all >>>>'", '...'
  hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
  hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
  bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}


def aiBuildButtonClick (_):
  # defaults
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()
  plid  = [courses[all] for all in courses][acIndex]
  video = playlists_yaml[plid]['videos'][rbIndex]
  vid   = video['url'].split('?v=')[-1]
  
  # del old buttons
  for oldbut in bx_csel.children: del oldbut
  
  # make new buttons
  tmp              = prettyMaking(enTrans[vid]).split('\n')
  lastTS           = 0
  chapters         = makeAutoChapters (int(hb_Caut.children[0].value),tmp)
  bu_desc.tooltip  = str(videoInfos[vid]['description'])
  bx_csel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                              tooltip      = c,
                              layout       = Layout(width='auto', height='21px')) 
                      for i,c in enumerate (chapters)]
  
  # wait message and start loop over chapters
  ta_entr.value, ta_xxtr.value = "... please wait - chapter building in progress >>>>'", '...'
  for b in bx_csel.children:
    start, end = getStartEndFromChapterButton (b)
    # get ai chapter title
    tmp = []
    if use_english_for_ai_auto_chapters:
      tmp   = taCleaner (prettyMaking ([t for t in enTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
      title = get_completion ( f"{prompts_yaml['aichapters']['en']}\n{tmp}", temperature=float(hb_encf.children[3].value) )
    else:
      tmp   = taCleaner (prettyMaking ([t for t in xxTrans[vid] if t['start'] >= start and t['start'] < end]).split('\n'),0)
      title = get_completion (f"{prompts_yaml['aichapters']['xx']}\n{tmp}", temperature=float(hb_xxcf.children[3].value))

    # set new title as button description
    if len (title) > 0:
      if title[0] == '"' or title[0] == "'":   title = title[1:]
      if title[-1] == '"' or title[-1] == "'": title = title[:-1]
    b.description = b.description.split(' ')[0]+' '+title
  for button in bx_csel.children: button.on_click(buttonClick)

  # unselect prompt / clear textareas
  ta_entr.value, ta_xxtr.value = "... select chapter or click 'all >>>>'", '...'
  hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
  hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
  bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}

  # store in yaml
  videoInfos[vid]['aichapters'] = [button.description for button in bx_csel.children]
  yaml.dump (videoInfos[vid], open(video['info'], 'w'))


#########################################################select video###############################################################
# show selected - build widgets for video - create video yamls
videoInfos, enTrans, xxTrans  = {}, {}, {}
def selectVideo (_):
  global mainAccordion
  # check help / prompt editor
  if mainAccordion.selected_index == len(mainAccordion.children) - 1 or mainAccordion.selected_index == len(mainAccordion.children) - 2:
    return

  # selection infos and the active widgets stored global
  gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
  bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
  bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()

  # get video infos for selected video - using acIndex and rbIndex
  print ('\r\x1b[35mplease wait a moment ...',end='')
  plid  = [courses[all] for all in courses][acIndex]
  video = playlists_yaml[plid]['videos'][rbIndex]
  vid   = video['url'].split('?v=')[-1]

  # create info yaml if not exists
  if not os.path.exists (video['info']):
    if not vid in videoInfos:
      videoInfo = Video.getInfo(video['url'])
      description = videoInfo['description']

      # parse chapter list / infoYml
      lines           = description.split('\n')
      chapters        = [l for l in lines if sum ([c == ':' or str(c).isdigit() for c in l.split(' ')[0]] ) == len(l.split(' ')[0])\
                                             and len(l.split(' ')[0]) > 0 and ':' in l.split(' ')[0] ]
      videoInfos[vid] = {'description':description, 'chapters':chapters}
    
    # write infoYml
    yaml.dump (videoInfos[vid], open(video['info'], 'w'))
    print (f"\r\x1b[34mwrite {video['info']}",end=' | ')

  # load infoYml
  else:
    if not vid in videoInfos:
      videoInfos[vid] = yaml.load(open(f"{video['info']}", 'r'), Loader=yaml.FullLoader)
      print (f"\r\x1b[34mload {video['info']}",end=' | ')
    else: print (f"\r\x1b[34mmemory {video['info']}",end=' | ')

  # create en transcription if not exists
  if not os.path.exists (video['trans_en']):
    # get transcriptions from youtube / write to yaml
    enTrans[vid] =  YouTubeTranscriptApi.get_transcript (vid)
    yaml.dump (enTrans[vid], open(f'{vid}_en.yml', 'w'))
  # load en transcription
  else:
    if not vid in enTrans:
      enTrans[vid] = enYml = yaml.load(open(f"{video['trans_en']}", 'r'), Loader=yaml.FullLoader)
      print (f"\x1b[34mload {video['trans_en']}",end=' | ')
    else: print (f"\x1b[34mmemory {video['trans_en']}",end=' | ')

  # create xx transcription if not exists
  if not os.path.exists (video['trans_xx']):
    # get transcriptions from youtube / transcriptions-dict / write to yaml
    cc = country_code_for_the_translation_language[:2]
    xxTrans[vid] = YouTubeTranscriptApi.list_transcripts (vid).find_generated_transcript([cc, 'en']).translate(cc).fetch()
    yaml.dump (xxTrans[vid], open(f'{vid}_{cc}.yml', 'w'))
  # load xx transcription
  else:
    if not vid in xxTrans:
      xxTrans[vid] = xxYml = yaml.load(open(f"{video['trans_xx']}", 'r'), Loader=yaml.FullLoader)
      print (f"\x1b[34mload {video['trans_xx']}",end='')
    else: print (f"\x1b[34mmemory {video['trans_xx']}",end='')

  # build video specific widgets
  chapters = videoInfos[vid]['chapters']
  bu_desc.tooltip = str(videoInfos[vid]['description'])
  # only if the video changed inside a playlist box
  if vPositions[acIndex] != rbIndex:
    # unselect prompt / clear textareas / color aichaps-button / message / lastState
    hb_encf.children[1].value, bu_enSh.disabled, bu_enSe.disabled, bu_enLo.disabled = '<<<select prompt>>>', True, True, True
    hb_xxcf.children[1].value, bu_xxSh.disabled, bu_xxSe.disabled, bu_xxLo.disabled = '<<<select prompt>>>', True, True, True
    bu_Cprm.style={'button_color':'pink'} if 'aichapters' in videoInfos[vid] else {'button_color':'lightcoral'}
    ta_entr.value, ta_xxtr.value = "... select chapter or click 'all >>>>'", '...'
    lastState = acIndex, rbIndex

    # new buttons
    for child in bx_csel.children: del child
    bx_csel.children = [Button (description  = f'{c:.67}...' if len (c) > 70 else c,
                                tooltip      = c,
                                layout       = Layout(width='auto', height='21px')) 
                        for i,c in enumerate (chapters)]
    for button in bx_csel.children: button.on_click(buttonClick)
    vPositions[acIndex] = rbIndex


#############################################################main###################################################################
# makes current widgets global
def globalize ():
  global vPositions, videoInfos, enTrans, xxTrans, courses, mainAccordion, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_xxcf, hb_Caut, bu_Cprm, mainLayout
  global hb_encf, bu_xxSe, bu_xxSh, bu_enSe, bu_enSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, bu_desc, rb_vids, hm_link, acIndex, rbIndex, bu_Cdes, bu_Caut, bu_Tcle, hb_Tcle
  global prompt_buttons, rb_prompts, bu_addprmt, bu_delprmt, bu_savprmt, hb_buttbar, tx_promtit, vb_promptedit, fastLayout, gb_pred
  gb_main                   = mainAccordion.children[mainAccordion.selected_index]
  vb_vsel, vb_chap          = gb_main.children[0], gb_main.children[1]
  bx_csel, hb_chcf, vb_view = vb_chap.children[0], vb_chap.children[1], vb_chap.children[2]
  hb_encf, hb_xxcf          = hb_chcf.children[0], hb_chcf.children[1]
  bu_xxSh, bu_xxSe          = hb_xxcf.children[4], hb_xxcf.children[5]
  bu_enSh, bu_enSe          = hb_encf.children[4], hb_encf.children[5]
  bu_enLo, bu_xxLo          = hb_encf.children[6], hb_xxcf.children[6]
  ta_entr, ta_xxtr          = vb_view.children[0], vb_view.children[1]
  hm_link, rb_vids, bu_desc = vb_vsel.children[0], vb_vsel.children[1], vb_vsel.children[2]
  bu_Cdes, hb_Caut, hb_Tcle = vb_vsel.children[4], vb_vsel.children[5], vb_vsel.children[3]
  hb_Cprm, bu_Crbl          = vb_vsel.children[6], vb_vsel.children[6].children[1] 
  bu_Cprm, bu_Caut, bu_Tcle = hb_Cprm.children[0], hb_Caut.children[1], hb_Tcle.children[1]
  acIndex, rbIndex          = mainAccordion.selected_index, rb_vids.options.index(rb_vids.value)
  return gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
         bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
         bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex

# prompt editor widgets
prompt_buttons            = [Button (description  =p,
                                     tooltip      =str(prompts_yaml[p]['en'])+'\n'+str(prompts_yaml[p]['xx']),
                                     style        ={'button_color':'lightgreen'}) for p in prompts_yaml if p != 'aichapters']
rb_prompts, bu_addprmt    = RadioButtons(options=[p for p in prompts_yaml]), Button (description='add prompt') 
bu_delprmt, bu_savprmt    = Button (description='del prompt'), Button (description='save') 
hb_buttbar, tx_promtit    = HBox (children=[bu_addprmt,bu_delprmt,bu_savprmt]), Text (layout=Layout(width='auto'))
hb_prompts                = HBox (children=[Textarea(layout=Layout(width='50%',height='400px')),
                                            Textarea(layout=Layout(width='50%',height='400px'))])
vb_promptedit             = VBox (children=[hb_buttbar,tx_promtit,hb_prompts])
fastLayout                = Layout (grid_template_rows='auto', grid_template_columns='150px auto', grid_template_areas='"sidebar main"')
gb_pred                   = GridBox(children=[rb_prompts,vb_promptedit],layout=fastLayout)
# events
for b in prompt_buttons: b.on_click (promptButtonClick)
bu_addprmt                .on_click (prompt_add)
bu_savprmt                .on_click (prompt_save)
bu_delprmt                .on_click (prompt_del)
rb_prompts                .observe  (promptSelect,names=['value'])
# show first
promptSelect(None)

# help tab
ta_help = Textarea(value=helptext.replace('\n*','\n\n*'),layout=Layout(width='auto',height='600px'))

# build and config widgets 
mainLayout    = Layout    (grid_template_rows='auto', grid_template_columns='150px auto', grid_template_areas='"sidebar main"')
mainAccordion = Accordion (children = [GridBox(layout=mainLayout) for all in courses] )
mainAccordion.children = tuple([*mainAccordion.children,gb_pred,ta_help])
mainAccordion.set_title(len(mainAccordion.children)-2,'prompt editor')
mainAccordion.set_title(len(mainAccordion.children)-1,'help')
# the pos of selected video inside a playlist box
vPositions = len(courses)*[-1]
# loop to build playlist related widgets structure
for i,title in enumerate(courses):
  # the widgets for a play list
  mainAccordion.set_title(i,f'Playlist: {title}')
  pl      = playlists_yaml[courses[title]]
  gb_main = mainAccordion.children[i]
  hm_link = HTML          (f'<a href="{playlist_prefix}{courses[title]} title="{"test"}">youtube</a>')
  rb_vids = RadioButtons  (options=[v['title'] for v in pl['videos']],layout=Layout(height='300px',overflow='scroll'))
  bu_desc = Button        (description='all >>>>',tooltip='',layout=Layout(width='auto'))
  bu_Cdes = Button        (description='from description',tooltip='Try parsing chapters out of video description.',
                           layout=Layout(width='auto'),style={'button_color':'pink'})
  bu_Caut = Button        (description='auto',tooltip='Division of the video into sections with max. tokens.',
                           layout=Layout(width='60%'),style={'button_color':'pink'})
  hb_Caut = HBox          (children=[Text (value='500',layout=Layout(width='40%')),bu_Caut])
  bu_Cprm = Button        (description='from ai',tooltip='Show the ai generated chapters',
                           layout=Layout(width='50%'),style={'button_color':'lightcoral'})
  bu_Crbl = Button        (description='aichaps',tooltip='(Re)build the ai chapters - use the token len from auto for chapter len.',
                           layout=Layout(width='50%'),style={'button_color':'lightcoral'})
  hb_Cprm = HBox          (children=[bu_Cprm,bu_Crbl])
  bu_Tcle = Button        (description='clean',tooltip='Comma seperated the number of generated timestamps and the max line lenght.',
                           layout=Layout(width='60%'))
  hb_Tcle = HBox          (children=[Text (value='3, 80',layout=Layout(width='40%')),bu_Tcle])
  vb_vsel = VBox          (children=[hm_link,rb_vids,bu_desc,hb_Tcle,bu_Cdes,hb_Caut,hb_Cprm,*prompt_buttons],layout=Layout(overflow="hidden"))
  bx_csel = Box           (layout=Layout(display='flex', flex_flow='wrap'))
  ta_entr = Textarea      (layout=Layout(width='50%',height='400px'))
  ta_xxtr = Textarea      (layout=Layout(width='50%',height='400px'))
  vb_view = HBox          (children=[ta_entr,ta_xxtr])
  bu_enSh = Button        (description='show',disabled=True,style={'button_color':'lightgreen'},layout=Layout(width='8%'))
  bu_enSe = Button        (description='send',disabled=True,button_style='warning',layout=Layout(width='8%'))
  bu_enLo = Button        (description='loop',disabled=True,button_style='warning',layout=Layout(width='8%'))
  bu_xxSh = Button        (description='show',disabled=True,style={'button_color':'lightgreen'},layout=Layout(width='8%'))
  bu_xxSe = Button        (description='send',disabled=True,button_style='warning',layout=Layout(width='8%'))
  bu_xxLo = Button        (description='lopp',disabled=True,button_style='warning',layout=Layout(width='8%'))
  hb_encf = HBox(children=[HTML(value='',layout=Layout(width='10%')),
                          Label(value='<<<select prompt>>>',layout=Layout(width='26%')),
                          Text(description='*',value='3.1',layout=Layout(width='20%')),
                          Text(description='°',value='0.3',layout=Layout(width='20%')),
                          bu_enSh, bu_enSe, bu_enLo], layout=Layout(border='1px solid black',width='50%'))
  hb_xxcf = HBox(children=[HTML(value='',layout=Layout(width='10%')),
                           Label(value='<<<select prompt>>>',layout=Layout(width='26%')),
                           Text(description='*',value='3.1',layout=Layout(width='20%')),
                           Text(description='°',value='0.3',layout=Layout(width='20%')),
                           bu_xxSh, bu_xxSe, bu_xxLo], layout=Layout(border='1px solid black',width='50%'))
  hb_chcf = HBox          (children=[hb_encf,hb_xxcf])
  vb_chap = VBox          (children=[bx_csel,hb_chcf,vb_view])
  gb_main.children        = [vb_vsel,vb_chap]
  # events
  rb_vids                 .observe  (selectVideo, names=['value'])
  mainAccordion           .observe  (selectVideo, names=['selected_index'])
  bu_desc                 .on_click (buttonClick)
  bu_enSh                 .on_click (enShowButtonClick)
  bu_xxSh                 .on_click (xxShowButtonClick)
  bu_enSe                 .on_click (enSendButtonClick)
  bu_xxSe                 .on_click (xxSendButtonClick)
  bu_enLo                 .on_click (enLoopButtonClick)
  bu_xxLo                 .on_click (xxLoopButtonClick)
  bu_Caut                 .on_click (autoButtonClick)
  bu_Cdes                 .on_click (fromDescriptionButtonClick)
  bu_Tcle                 .on_click (cleanButtonClick)
  bu_Cprm                 .on_click (aiShowButtonClick)
  bu_Crbl                 .on_click (aiBuildButtonClick)
  ta_entr                 .observe  (text_change, names=['value'])
  ta_xxtr                 .observe  (text_change, names=['value'])

#display / globalize current displayed / set first selection
display(mainAccordion)
gb_main, vb_vsel, vb_chap, bx_csel, hb_chcf, vb_view, hb_encf, hb_xxcf, bu_enSh,\
bu_enSe, bu_xxSh, bu_enLo, bu_xxLo, ta_entr, ta_xxtr, hm_link, rb_vids, bu_desc, hb_Tcle,\
bu_Cdes, hb_Caut, hb_Cprm, bu_Crbl, bu_Cprm, bu_Caut, bu_Tcle, acIndex, rbIndex = globalize ()

selectVideo(None)


